# Homework week 5

## Q1

* Install Pipenv 
* What's the version of pipenv you installed? 
* Use --version to find out

##from terminal

pip install pipenv
pipenv --version



## Q2

* Use Pipenv to install Scikit-Learn version 1.0  
* What's the first hash for scikit-learn you get in Pipfile.lock?

#from terminal

pipenv install scikit-learn==1.0

"sha256:121f78d6564000dc5e968394f45aac87981fcaaf2be40cfcd8f07b2baa1e1829"

To activate the environment run the following: pipenv shell

## Q3

Let's use these models!  
* Write a script for loading these models with pickle 
* Score this customer:

{"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}

What's the probability that this customer is churning?

In [2]:
import pickle

def load(filename):
    with open(filename, 'rb') as f_in:
        return pickle.load(f_in)

In [3]:
#load files from pickle
dv = load('dv.bin')
model = load('model1.bin')

In [4]:
#score a customer
customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}

X = dv.transform([customer])
y_pred = model.predict_proba(X)[0, 1]

print(y_pred)

0.11549580587832914


## Q4

Now let's serve this model as a web service

* Install Flask and Gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this customer using requests:

url = "YOUR_URL" 

customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10} 

requests.post(url, json=customer).json()

#from terminal

pipenv install flask
pipenv install gunicorn

#write flask app: see q4_predict.py 

#execute app from terminal: python q4_predict.py 
##or do it via gunicorn to avoid "in production" warning: gunicorn --bind 0.0.0.0:9696 q4_predict:app

To score the customer, you can execute the following code via  JUPYTER NOTEBOOK:

see q4_predict_test.ipynb

## Docker

Install Docker. We will use it for the next two questions.

For these questions, I prepared a base image: agrigorev/zoomcamp-model:3.8.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.8.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

FROM python:3.8.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]

## Q5

1) Install docker container prepared for the exercise as follows: docker run -it --rm agrigorev/zoomcamp-model:3.8.12-slim
#if we run the above we enter by default a python terminal. To enter a bash terminal run as follows: docker run -it --rm --entrypoint=bash agrigorev/zoomcamp-model:3.8.12-slim

2) Create "Dockerfile" an prepare it as per instructions

3) Build dockerfile as follows: docker build -t homework_w5 .
#the dot is to tell to use the Dockerfile present in the current directory
#it will run all the commands inside the Dockerfile and install required packages

4) Now we can finally run the newly created image as we did earlier, as follows: docker run -it --rm --entrypoint=bash homework_w5

5) If we now run ls we can see that all files mentioned in the Dockerfile are there: 
#Pipfile  Pipfile.lock  dv.bin  model2.bin  q6_predict.py

6) Run it with gunicorn: gunicorn --bind=0.0.0.0:9696 q6_predict:app

#we can see that it's listening at port e.g. http://0.0.0.0:9696 though we can't yet access this port because we have first to expose this port.We have to say Docker "Hey Docker we want to have this port open for the host machine...". And also we need to do a port mapping, to map the port we have in the contained to the port in our host machine... so the predict.py will point to the host port and this will be mapped to a port on the Docker machine and then it will go to the Churn service, and then our serive will reply with the prediction and etc. etc. the inverse path. This is done by adding this line in the Docker file: EXPOSE 9696 and specify entrypoints: ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "q6_predict:app"]

#to get the is of the image we can run the following outise of docker terminal: docker images
#in the IMAGE ID column you can find the id e.g. dc3bae430933   


## Q6

Let's run your docker container!

After running it, score this customer:

url = "YOUR_URL"

customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 10}

requests.post(url, json=customer).json()

What's the probability that this customer is churning?

We can now use again our Jupyter notebook q4_predict_test to make the prediction request.